In [2]:
import pandas as pd
import numpy as np
import os, sys
sys.path.insert(0, os.path.abspath('../..'))
import source.data_preparation as dp
from source.configs_gw import WITHOUT_LOOKAHEAD, ALL, MINIMAL_SETTING, WITHOUT_LOOKAHEAD_AR, ALL_COVARIATES_NON_AR, ALL_COVARIATES_AR,COMPLETED
import source.Bootstrapping.models_regression as bu
import source.regression_models as modl
data = dp.prepare_data(uselog=True)

/opt/anaconda3/envs/tsfm/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
# Example: TabPFN on WITHOUT_LOOKAHEAD with lag=1
df_lag = bu.make_lagged_features(data, COMPLETED, lag=1)
feature_cols = [f"{v}_lag1" for v in COMPLETED]


In [28]:
df_lag[feature_cols]

,lzrt_lag1,wtexas_lag1,skvw_lag1,tail_lag1,dtoy_lag1,dtoat_lag1,rdsp_lag1,avgcor_lag1,ntis_lag1,tbl_lag1,...,e/p_lag1,d/e_lag1,svar_lag1,lty_lag1,ltr_lag1,tms_lag1,dfy_lag1,dfr_lag1,infl_lag1,b/m_lag1
date,,,,,,,,,,,,,,,,,,,,,
1871-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.090090,0.650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.088889,0.650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.086768,0.650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.084388,0.650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-01,-1.918444,-0.041893,-0.006445,0.484421,0.991376,0.991376,0.049683,0.112621,-0.011832,0.0520,...,0.035742,0.367068,0.001748,0.0425,0.021879,-0.0095,0.0072,0.001956,0.001162,0.190681
2024-09-01,-1.995251,-0.060988,-0.087008,0.498976,1.000000,1.000000,0.034581,0.195877,-0.011028,0.0505,...,0.035200,0.366785,0.003184,0.0387,0.012792,-0.0118,0.0073,0.002934,0.000814,0.187377
2024-10-01,-1.915717,-0.077429,-0.035656,0.480061,1.000000,1.000000,0.023585,0.236572,-0.010283,0.0472,...,0.034754,0.366507,0.001466,0.0372,0.012023,-0.0100,0.0074,0.005707,0.001604,0.183981


In [32]:
from importlib import reload
reload(bu)

<module 'source.Bootstrapping.models_regression' from '/Users/kilianrunnwerth/Masterarbeit/source/Bootstrapping/models_regression.py'>

In [30]:
# PCR
stats = bu.bootstrap_refit_train_test(
    df_lag,
    target_col="equity_premium",
    feature_cols=feature_cols,
    start_oos="1927-01-01",
    min_train=240,
    B=1000,
    seed=42,
    bootstrap="iid",
    model="pcr",
    pcr_k=1,
    ct_cutoff=True,
)
print(stats)

Bootstrap 1/1000: df_test has 431/1176 observations (36.65%)
Bootstrap 2/1000: df_test has 435/1176 observations (36.99%)
Bootstrap 3/1000: df_test has 424/1176 observations (36.05%)
Bootstrap 4/1000: df_test has 425/1176 observations (36.14%)
Bootstrap 5/1000: df_test has 411/1176 observations (34.95%)
Bootstrap 6/1000: df_test has 429/1176 observations (36.48%)
Bootstrap 7/1000: df_test has 439/1176 observations (37.33%)
Bootstrap 8/1000: df_test has 420/1176 observations (35.71%)
Bootstrap 9/1000: df_test has 436/1176 observations (37.07%)
Bootstrap 10/1000: df_test has 442/1176 observations (37.59%)
Bootstrap 11/1000: df_test has 432/1176 observations (36.73%)
Bootstrap 12/1000: df_test has 435/1176 observations (36.99%)
Bootstrap 13/1000: df_test has 429/1176 observations (36.48%)
Bootstrap 14/1000: df_test has 430/1176 observations (36.56%)
Bootstrap 15/1000: df_test has 427/1176 observations (36.31%)
Bootstrap 16/1000: df_test has 434/1176 observations (36.90%)
Bootstrap 17/1000

In [33]:
# PCR
stats = bu.bootstrap_refit_train_test(
    df_lag,
    target_col="equity_premium",
    feature_cols=feature_cols,
    start_oos="1927-01-01",
    min_train=240,
    B=1000,
    seed=42,
    bootstrap="iid",
    model="combo",
    ct_cutoff=True,
)
print(stats)

Bootstrap 1/1000: df_test has 431/1176 observations (36.65%)
Bootstrap 2/1000: df_test has 435/1176 observations (36.99%)
Bootstrap 3/1000: df_test has 424/1176 observations (36.05%)
Bootstrap 4/1000: df_test has 425/1176 observations (36.14%)
Bootstrap 5/1000: df_test has 411/1176 observations (34.95%)
Bootstrap 6/1000: df_test has 429/1176 observations (36.48%)
Bootstrap 7/1000: df_test has 439/1176 observations (37.33%)
Bootstrap 8/1000: df_test has 420/1176 observations (35.71%)
Bootstrap 9/1000: df_test has 436/1176 observations (37.07%)
Bootstrap 10/1000: df_test has 442/1176 observations (37.59%)
Bootstrap 11/1000: df_test has 432/1176 observations (36.73%)
Bootstrap 12/1000: df_test has 435/1176 observations (36.99%)
Bootstrap 13/1000: df_test has 429/1176 observations (36.48%)
Bootstrap 14/1000: df_test has 430/1176 observations (36.56%)
Bootstrap 15/1000: df_test has 427/1176 observations (36.31%)
Bootstrap 16/1000: df_test has 434/1176 observations (36.90%)
Bootstrap 17/1000

In [ ]:

r2_point, stats = bu.bootstrap_refit_train_test(
    df_lag,
    target_col="equity_premium",
    feature_cols=feature_cols,
    start_oos="1957-01-01",
    min_train=240,
    B=1000,
    seed=42,
    bootstrap="iid",   # or "mbb"
    block_size=12,
    model="tabpfn",
    ct_cutoff=True,
)
print(r2_point, stats["mean"], stats["ci_lower"], stats["ci_upper"])
# OLS combination regression (mean/median/trimmed_mean)



In [7]:
_, _, _, y_pred_m, _, _, _, HA_m = modl.chronos2_oos(
    data[data.index >= "1927-01-01"], covariates=[], start_oos="1957-01-01",
    ctx_min=256, ct_cutoff=True, quiet=False, mode = "mean", freq="M"
)
stats = bu.bootstrap_refit_train_test(
    df_lag,
    target_col="equity_premium",
    feature_cols=feature_cols,
    start_oos="1957-01-01",
    min_train=240,
    B=1000,
    seed=42,
    bootstrap="mbb",  # recommended for ARIMA
    block_size=12,
    model="chronos_2",
    ct_cutoff=True,
    y_pred_given=y_pred_m,
)
stats

`torch_dtype` is deprecated! Use `dtype` instead!


[Chronos-2] Loading amazon/chronos-2 on M freq with 0 covariates...


Loading weights:   0%|          | 0/170 [00:00<?, ?it/s]

1957-01-01 00:00:00


/opt/anaconda3/envs/tsfm/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


1957-02-01 00:00:00
1957-03-01 00:00:00
1957-04-01 00:00:00
1957-05-01 00:00:00
1957-06-01 00:00:00
1957-07-01 00:00:00
1957-08-01 00:00:00
1957-09-01 00:00:00
1957-10-01 00:00:00
1957-11-01 00:00:00
1957-12-01 00:00:00
1958-01-01 00:00:00
1958-02-01 00:00:00
1958-03-01 00:00:00
1958-04-01 00:00:00
1958-05-01 00:00:00
1958-06-01 00:00:00
1958-07-01 00:00:00
1958-08-01 00:00:00
1958-09-01 00:00:00
1958-10-01 00:00:00
1958-11-01 00:00:00
1958-12-01 00:00:00
1959-01-01 00:00:00
1959-02-01 00:00:00
1959-03-01 00:00:00
1959-04-01 00:00:00
1959-05-01 00:00:00
1959-06-01 00:00:00
1959-07-01 00:00:00
1959-08-01 00:00:00
1959-09-01 00:00:00
1959-10-01 00:00:00
1959-11-01 00:00:00
1959-12-01 00:00:00
1960-01-01 00:00:00
1960-02-01 00:00:00
1960-03-01 00:00:00
1960-04-01 00:00:00
1960-05-01 00:00:00
1960-06-01 00:00:00
1960-07-01 00:00:00
1960-08-01 00:00:00
1960-09-01 00:00:00
1960-10-01 00:00:00
1960-11-01 00:00:00
1960-12-01 00:00:00
1961-01-01 00:00:00
1961-02-01 00:00:00
1961-03-01 00:00:00


{'mean': 0.0018338471672407332,
 'std': 0.005886881481983248,
 'ci_lower': -0.0038206733000070606,
 'ci_upper': 0.013427754005349872,
 'ci': 0.9,
 'n': 1000}